In [3]:
import pandas as pd

activities = pd.read_csv("C:/Users/anhin/Documents/GitHub/MoonKEY/data/activitats.csv", encoding='ISO-8859-1')
marks = pd.read_csv("C:/Users/anhin/Documents/GitHub/MoonKEY/data/notes.csv", delimiter=';')
submision = pd.read_csv("C:/Users/anhin/Documents/GitHub/MoonKEY/data/trameses.csv")

pd.options.display.float_format = '{:,.0f}'.format

In [9]:
user_31 = submision[(submision["userid"] == 31) & (submision["activitat_id"] == 362)]
user_31 = user_31.fillna(-1)
user_31 = user_31.sort_values(by=['datesubmitted']).copy()
user_31['attempt_number'] = user_31.groupby('userid').cumcount() + 1
user_31 = user_31[user_31["grade"] == max(user_31["grade"]) & user_31["dategraded"] == min(user_31["dategraded"])]



TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]

In [10]:
user_31

,id,activitat_id,userid,datesubmitted,grader,dategraded,grade,nevaluations,attempt_number
3,15,362,31,1662537577,0,"1,662,537,582",-1,0,1
4,16,362,31,1662537593,0,"1,662,712,109",100,0,2
7,19,362,31,1662712258,0,"1,662,712,261",100,0,3
8,20,362,31,1662712268,0,"1,662,712,272",0,0,4
11,23,362,31,1662971060,0,"1,662,971,064",100,0,5
80,141,362,31,1662993906,0,0,-1,0,6
90,157,362,31,1662994016,0,"1,662,994,038",100,0,7
92,165,362,31,1662994071,0,"1,662,994,074",0,0,8
93,169,362,31,1662994111,0,"1,662,994,114",0,0,9
96,179,362,31,1662994190,0,"1,662,994,194",0,0,10


In [ ]:
# Convertir timestamp a formato de fecha
submision['datesubmitted'] = pd.to_datetime(submision['datesubmitted'], unit='s')
print(submision['datesubmitted'])
